In [13]:
import torch
import torch.optim as optim
from pytorch_efficientnet import EfficientNetwork
from src.train_test_functions import train_model
import json
import numpy as np
from src.torch_dataset import ZootrDataset 
import os
from pytorch_efficientnet import to_tensor, EfficientNetwork, eval_model, augment, augmented_pred

## Set constants

In [14]:
BATCH_SIZE = 20
NUM_WORKERS = 0
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.1
EPOCHS = 10

In [15]:
with open('config/paths_dict.json') as handle:
    paths_dict = json.loads(handle.read())

ITEM_PATH = paths_dict['item_path']
NON_ITEM_PATH = paths_dict['non_item_path']

In [19]:
load_data_objects = False
save_data_objects = True

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Set up data loaders

In [20]:
if  load_data_objects:
    torch.load('Models/train_data.pth')
    torch.load('Models/valid_data.pth')
else:
    items_images = [ITEM_PATH + i for i in os.listdir(ITEM_PATH)]
    #Get all items
    non_items_images = [NON_ITEM_PATH + i for i in os.listdir(NON_ITEM_PATH)]
    #Get non items
    all_images = list(np.random.permutation(items_images + non_items_images))
    #Combine all images' paths and shuffle
    train_data = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augment)
    train_data_eval = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augmented_pred)
    valid_data = ZootrDataset(all_images[int(np.ceil(len(all_images)*0.8)):], transform_=augmented_pred)
    # Data Objects
    if save_data_objects:
        torch.save(train_data, 'Models/train_data.pth')
        torch.save(valid_data, 'Models/valid_data.pth')

## Model

In [22]:
model = EfficientNetwork(output_size=2, b1=False, b2=True)
# Criterion
criterion = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',
                                                       patience=1, factor=0.4)

Loaded pretrained weights for efficientnet-b2


In [ ]:
dics = {}
dics['dataloaders'] = dataloaders
dics['dataset_sizes'] = dataset_sizes

In [ ]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x1dc2a47d290>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x1dc29d32e50>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x1dc2977b950>}

In [ ]:
mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=25)

Starting  train
Epoch 1/25. Batch 1/44
Epoch 1/25. Batch 2/44
Epoch 1/25. Batch 3/44
Epoch 1/25. Batch 4/44
Epoch 1/25. Batch 5/44
Epoch 1/25. Batch 6/44
Epoch 1/25. Batch 7/44
Epoch 1/25. Batch 8/44
Epoch 1/25. Batch 9/44
Epoch 1/25. Batch 10/44
Epoch 1/25. Batch 11/44
Epoch 1/25. Batch 12/44
Epoch 1/25. Batch 13/44
Epoch 1/25. Batch 14/44
Epoch 1/25. Batch 15/44
Epoch 1/25. Batch 16/44
Epoch 1/25. Batch 17/44
Epoch 1/25. Batch 18/44
Epoch 1/25. Batch 19/44
Epoch 1/25. Batch 20/44
Epoch 1/25. Batch 21/44
Epoch 1/25. Batch 22/44
Epoch 1/25. Batch 23/44
Epoch 1/25. Batch 24/44
Epoch 1/25. Batch 25/44
Epoch 1/25. Batch 26/44
Epoch 1/25. Batch 27/44
Epoch 1/25. Batch 28/44
Epoch 1/25. Batch 29/44
Epoch 1/25. Batch 30/44
Epoch 1/25. Batch 31/44
Epoch 1/25. Batch 32/44
Epoch 1/25. Batch 33/44
Epoch 1/25. Batch 34/44
Epoch 1/25. Batch 35/44
Epoch 1/25. Batch 36/44
Epoch 1/25. Batch 37/44
Epoch 1/25. Batch 38/44
Epoch 1/25. Batch 39/44
Epoch 1/25. Batch 40/44
Epoch 1/25. Batch 41/44
Epoch 1/2

In [ ]:
test_loader = dataloaders['test']

all_preds = torch.empty(0, 2)
all_labels = []
for imgs, label in test_loader:
    imgs_pred = mdl(imgs)

    all_preds = torch.cat((all_preds, imgs_pred))
    all_labels += label

In [ ]:
labels = [i.item() for i in all_labels]

In [ ]:
import pandas as pd

preds_prob = torch.softmax(all_preds, dim=1)
preds_prob = pd.DataFrame(preds_prob.detach().numpy())
preds_prob.columns = ['False', 'True']
preds_prob['Label'] = labels

In [ ]:
preds_prob.sort_values('True', ascending=False)

,False,True,Label
39,0.000055,0.999945,1
217,0.000083,0.999917,1
92,0.000093,0.999907,1
229,0.000103,0.999897,1
26,0.000219,0.999781,1
...,...,...,...
4,0.999955,0.000045,0
201,0.999962,0.000038,0
33,0.999964,0.000035,0
221,0.999971,0.000029,0


In [ ]:
from sklearn.metrics import roc_auc_score

preds_prob[preds_prob['Label'] == 1].sort_values('True', ascending=False)
roc_auc_score(preds_prob['Label'], preds_prob['True'])

0.9625237886107451

In [ ]:
import numpy as np

print(np.mean(preds_prob.loc[preds_prob['True'] >= 0.9, 'Label']))
print(np.mean(preds_prob.loc[preds_prob['True'] < 0.9, 'Label']))

0.8888888888888888
0.04225352112676056


In [ ]:
#torch.save(mdl, 'model.pth')